In [1]:
import pandas as pd
import gzip
import json


In [2]:
from tqdm import tqdm

In [3]:
subset = "Beauty_and_Personal_Care"
nametofile = "amazon_"+subset

In [4]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in tqdm(parse(path)):
    df[i] = d
    i += 1
    if i>7000000:
        break
  return pd.DataFrame.from_dict(df, orient='index')
    
def getDF2(path):
  i = 0
  df = {}
  for d in tqdm(parse(path)):
    i += 1
    if i<=7000000:
        continue
    df[i] = d
    
  return pd.DataFrame.from_dict(df, orient='index')


In [5]:
df = getDF(f'./data/{subset}.jsonl.gz')

7000000it [01:32, 75598.97it/s] 


In [ ]:
df2 = getDF2(f'./data/{subset}.jsonl.gz')

11394462it [07:58, 96928.77it/s] 

In [6]:
df = df.dropna()

In [7]:
new_df = df[["title", "text", "asin", "user_id"]]


In [8]:
new_df

,title,text,asin,user_id
0,Excellent! Yummy!,Excellent!! Yummy! Great with other foods and...,B00CM36GAQ,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ
1,Delicious!!! Yum!,Excellent! The best! I use it with my beef a...,B074J5WVYH,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ
2,"Extremely Delicious, but expensive imo",These are very tasty. They are extremely soft ...,B079TRNVHX,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ
3,Delicious!,My favorite!,B07194LN2Z,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ
4,Great taste,Great for making brownies and crinkle cookies.,B005CD4196,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ
...,...,...,...,...
6999996,Quite a bargain,"What's not to like? Shelled pistachios, very f...",B00K6FBMYM,AG7S5NL4OQQKBUG3TKAL76GHUPZQ
6999997,Super fast delivery,This is a good low acid coffee. I can't drink ...,B0815B75PX,AG7S5NL4OQQKBUG3TKAL76GHUPZQ
6999998,Tangy and not too hot,"This is my favorite hot sauce, because it is n...",B00B0NQ186,AG7S5NL4OQQKBUG3TKAL76GHUPZQ
6999999,"Good flavor, low acid","Very good flavor, but low acid. I really can't...",B073VV4GZB,AG7S5NL4OQQKBUG3TKAL76GHUPZQ


In [8]:
def checkmydf(mydf, name="asin"):
    num_unique_user_id = mydf["user_id"].nunique()
    num_unique_asin = mydf[name].nunique()
    
    # Print unique counts
    print("Number of unique user_id:", num_unique_user_id)
    print("Number of unique asin:", num_unique_asin)

In [10]:
checkmydf(new_df)

# Save the filtered DataFrame to a CSV file
new_df.to_csv(f"./data/reviews/filtered_data{nametofile}.csv", index=False)

print("Filtered DataFrame saved to 'filtered_data.csv'")

Number of unique user_id: 2254352
Number of unique asin: 569350
Filtered DataFrame saved to 'filtered_data.csv'


In [5]:
new_df = pd.read_csv(f"./data/reviews/filtered_data{nametofile}.csv")

In [29]:
asin_counts = new_df['asin'].value_counts()
valid_asins = asin_counts[asin_counts >= 20].index
df_filtered = new_df[new_df['asin'].isin(valid_asins)]

In [9]:
checkmydf(df_filtered)

Number of unique user_id: 2698416
Number of unique asin: 82590


In [ ]:
# unique_asin = df_filtered["asin"].unique()
# sampled_asins = pd.Series(unique_asin).sample(n=20000, random_state=1009)
# df_subset = df_filtered[df_filtered['asin'].isin(sampled_asins)]

In [ ]:
# def get_top_k_users(df, k):
#     return df['user_id'].value_counts().head(k).index.tolist()

In [ ]:
# listKu = get_top_k_users(df_filtered, 20000)

In [30]:
df_subset = df_filtered
# df_filtered = filtered_users_df
# df_subset = df_filtered[df_filtered['user_id'].isin(listKu)]


In [33]:
df_subset = df_subset.dropna()

In [34]:
df_subset 

,title,text,asin,user_id
0,Gasoline!! Seriously reeks of gasoline!,Opened the package & instant migraine. I canno...,B00Z03RC80,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ
1,Useless! These have massive gaps at pump nozzle.,Tops the list for worst purchase. Tried these ...,B085PRT2MP,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ
3,Not sharp enough to cut smoothly.,These are extremely dull and will wreck your n...,B07YYG76X1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ
4,"Pretty, but didn’t close correctly.",Mine wouldn’t close correctly so I sent it bac...,B0761M33BX,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ
5,Unbottled disappointment,I don't often leave negative feedback about pr...,B010TWN80W,AGGZ357AO26RQZVRLGU4D4N52DZQ
...,...,...,...,...
9999995,Small & Mighty,I LOVE this stuff. I use it on my lips nightly...,B073X9M2GL,AFI45FMG3632MRERWUA34T7IWO7Q
9999996,Not on par with other products,Is there any product in the tube? Literally sc...,B08RS8S9KP,AFI45FMG3632MRERWUA34T7IWO7Q
9999997,Big and fluffy,These are so big!!! I love them. They're super...,B078V5WPPZ,AFI45FMG3632MRERWUA34T7IWO7Q
9999998,Best cleanser out there,While I LOVE this face wash and I truly do lov...,B015OWD370,AFI45FMG3632MRERWUA34T7IWO7Q


In [35]:
user_review_counts = df_subset["user_id"].value_counts()
users_with_at_least_5_reviews = user_review_counts[user_review_counts >= 25].index
filtered_users_df = df_subset[df_subset["user_id"].isin(users_with_at_least_5_reviews)]
filtered_users_df.rename(columns={"asin": "rest_id"}, inplace=True)
filtered_users_df["text"] = filtered_users_df["title"] + ". " + filtered_users_df["text"]
filtered_users_df["review_id"] = filtered_users_df.index

asin_counts = filtered_users_df['rest_id'].value_counts()
valid_asins = asin_counts[asin_counts >= 25].index
filtered_users_df = filtered_users_df[filtered_users_df['rest_id'].isin(valid_asins)]


filtered_users_df.to_csv(f"./data/reviews/{nametofile}.csv", index=False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_13544\4036187543.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_users_df.rename(columns={"asin": "rest_id"}, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_13544\4036187543.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_users_df["text"] = filtered_users_df["title"] + ". " + filtered_users_df["text"]
C:\Users\Admin\AppData\Local\Temp\ipykernel_13544\4036187543.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [380]:
filtered_users_df = pd.read_csv(f"./data/reviews/{nametofile}.csv")

In [36]:
filtered_users_df

,title,text,rest_id,user_id,review_id
222,Bad smell,Bad smell. I brought this to shine and clump c...,B0030F0U3Y,AEYORY2AVPMCPDV57CE337YU5LXA,222
227,Buy it,Buy it. This was my first Denman brush. I brou...,B00197623M,AEYORY2AVPMCPDV57CE337YU5LXA,227
230,Okay,Okay. This makes your skin feel nice. I wish i...,B007DGRT3K,AEYORY2AVPMCPDV57CE337YU5LXA,230
236,Wow,Wow. I’ve never used cuticle oil before so I d...,B00F644L32,AEYORY2AVPMCPDV57CE337YU5LXA,236
238,Get it,Get it. I was concerned about how this would l...,B018YOVQSI,AEYORY2AVPMCPDV57CE337YU5LXA,238
...,...,...,...,...,...
9996341,Impressive Little Thing,Impressive Little Thing. I've been wanting to ...,B004DIUX34,AHSLUOV2N2UYNJYQAK7YYONCMPHQ,9996341
9996345,Works Great as Primer and to Control Oil,Works Great as Primer and to Control Oil. This...,B000GCQ04C,AHSLUOV2N2UYNJYQAK7YYONCMPHQ,9996345
9996346,My New HG,My New HG. I finally tried this product after ...,B002ZTVMDI,AHSLUOV2N2UYNJYQAK7YYONCMPHQ,9996346
9996347,4.5 Stars--Don't Pass This Up,"4.5 Stars--Don't Pass This Up. Overall, this l...",B002LFRRIK,AHSLUOV2N2UYNJYQAK7YYONCMPHQ,9996347


In [37]:
filtered_users_df['rating'] = 5
checkmydf(filtered_users_df, "rest_id")
# Save the modified DataFrame back to a new CSV file
filtered_users_df.to_csv(f"./data/reviews/{nametofile}.csv", index=False)

Number of unique user_id: 8363
Number of unique asin: 1434


In [38]:
finalAuth = list(filtered_users_df["user_id"].unique())

In [39]:
len(finalAuth)

8363

In [40]:
import random

percentage = 0.04
num_authors = int(len(finalAuth) * percentage)
test_authors = random.sample(finalAuth, num_authors)


In [41]:
remain_author = set(finalAuth) - set(test_authors)
percentage = 0.4
num_authors = int(len(remain_author) * percentage)
train_authors = random.sample(remain_author, num_authors)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13544\163831081.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  train_authors = random.sample(remain_author, num_authors)


In [42]:
print("Number of authors in training set:", len(train_authors))
print("Number of authors in testing set:", len(test_authors))

Number of authors in training set: 3211
Number of authors in testing set: 334


In [43]:
#  save train and test to json as following: {"amazon": {"train": ["PVyZXgOkVtnU6966FDFhuw",..], "test": ["PVyZXgOkVtnU6966FDFhuw",..]}}

data = {
    f"{nametofile}": {
        "train": list(train_authors),
        "test": list(test_authors)
    }
}

with open(f'./data/reviews/{nametofile}_splits.json', 'w') as f:
  json.dump(data, f)

In [44]:
s1 = set(filtered_users_df[filtered_users_df['user_id'].isin(train_authors)]['rest_id'].tolist())
s2 = set(filtered_users_df[filtered_users_df['user_id'].isin(test_authors)]['rest_id'].tolist())

In [45]:
len(s2-s1), len(s1), len(s2), len(s2|s1)

(0, 1434, 1077, 1434)

In [46]:
user_review_counts = filtered_users_df["rest_id"].value_counts()
user_review_counts.min()

np.int64(25)

# Refine data

In [2]:
import pandas as pd

In [180]:
asin_counts = filtered_users_df['rest_id'].value_counts()
valid_asins = asin_counts[asin_counts >= 20].index
df_filtered = filtered_users_df[filtered_users_df['rest_id'].isin(valid_asins)]